In [ ]:
%load_ext d2lbook.tab
tab.interact_select(['mxnet', 'pytorch', 'tensorflow', 'jax'])

# Parameter Management

Once we have chosen an architecture
and set our hyperparameters,
we proceed to the training loop,
where our goal is to find parameter values
that minimize our loss function.
After training, we will need these parameters
in order to make future predictions.
Additionally, we will sometimes wish
to extract the parameters
perhaps to reuse them in some other context,
to save our model to disk so that
it may be executed in other software,
or for examination in the hope of
gaining scientific understanding.

Most of the time, we will be able
to ignore the nitty-gritty details
of how parameters are declared
and manipulated, relying on deep learning frameworks
to do the heavy lifting.
However, when we move away from
stacked architectures with standard layers,
we will sometimes need to get into the weeds
of declaring and manipulating parameters.
In this section, we cover the following:

* Accessing parameters for debugging, diagnostics, and visualizations.
* Sharing parameters across different model components.

In [ ]:
%%tab mxnet
from mxnet import init, np, npx
from mxnet.gluon import nn
npx.set_np()

In [ ]:
%%tab pytorch
import torch
from torch import nn

In [ ]:
%%tab tensorflow
import tensorflow as tf

In [ ]:
%%tab jax
from d2l import jax as d2l
from flax import linen as nn
import jax
from jax import numpy as jnp

(**We start by focusing on an MLP with one hidden layer.**)

In [ ]:
%%tab mxnet
net = nn.Sequential()
net.add(nn.Dense(8, activation='relu'))
net.add(nn.Dense(1))
net.initialize()  # Use the default initialization method

X = np.random.uniform(size=(2, 4))
net(X).shape

In [ ]:
%%tab pytorch
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

In [ ]:
%%tab tensorflow
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X).shape

In [ ]:
%%tab jax
net = nn.Sequential([nn.Dense(8), nn.relu, nn.Dense(1)])

X = jax.random.uniform(d2l.get_key(), (2, 4))
params = net.init(d2l.get_key(), X)
net.apply(params, X).shape

## [**Parameter Access**]
:label:`subsec_param-access`

Let's start with how to access parameters
from the models that you already know.

:begin_tab:`mxnet, pytorch, tensorflow`
When a model is defined via the `Sequential` class,
we can first access any layer by indexing
into the model as though it were a list.
Each layer's parameters are conveniently
located in its attribute.
:end_tab:

:begin_tab:`jax`
Flax and JAX decouple the model and the parameters as you
might have observed in the models defined previously.
When a model is defined via the `Sequential` class,
we first need to initialize the network to generate
the parameters dictionary. We can access
any layer's parameters through the keys of this dictionary.
:end_tab:

We can inspect the parameters of the second fully connected layer as follows.

In [ ]:
%%tab mxnet
net[1].params

In [ ]:
%%tab pytorch
net[2].state_dict()

In [ ]:
%%tab tensorflow
net.layers[2].weights

In [ ]:
%%tab jax
params['params']['layers_2']

We can see that this fully connected layer
contains two parameters,
corresponding to that layer's
weights and biases, respectively.


### [**Targeted Parameters**]

Note that each parameter is represented
as an instance of the parameter class.
To do anything useful with the parameters,
we first need to access the underlying numerical values.
There are several ways to do this.
Some are simpler while others are more general.
The following code extracts the bias
from the second neural network layer, which returns a parameter class instance, and
further accesses that parameter's value.

In [ ]:
%%tab mxnet
type(net[1].bias), net[1].bias.data()

In [ ]:
%%tab pytorch
type(net[2].bias), net[2].bias.data

In [ ]:
%%tab tensorflow
type(net.layers[2].weights[1]), tf.convert_to_tensor(net.layers[2].weights[1])

In [ ]:
%%tab jax
bias = params['params']['layers_2']['bias']
type(bias), bias

:begin_tab:`mxnet,pytorch`
Parameters are complex objects,
containing values, gradients,
and additional information.
That is why we need to request the value explicitly.

In addition to the value, each parameter also allows us to access the gradient. Because we have not invoked backpropagation for this network yet, it is in its initial state.
:end_tab:

:begin_tab:`jax`
Unlike the other frameworks, JAX does not keep a track of the gradients over the
neural network parameters, instead the parameters and the network are decoupled.
It allows the user to express their computation as a
Python function, and use the `grad` transformation for the same purpose.
:end_tab:

In [ ]:
%%tab mxnet
net[1].weight.grad()

In [ ]:
%%tab pytorch
net[2].weight.grad == None

### [**All Parameters at Once**]

When we need to perform operations on all parameters,
accessing them one-by-one can grow tedious.
The situation can grow especially unwieldy
when we work with more complex, e.g., nested, modules,
since we would need to recurse
through the entire tree to extract
each sub-module's parameters. Below we demonstrate accessing the parameters of all layers.

In [ ]:
%%tab mxnet
net.collect_params()

In [ ]:
%%tab pytorch
[(name, param.shape) for name, param in net.named_parameters()]

In [ ]:
%%tab tensorflow
net.get_weights()

In [ ]:
%%tab jax
jax.tree_util.tree_map(lambda x: x.shape, params)

## [**Tied Parameters**]

Often, we want to share parameters across multiple layers.
Let's see how to do this elegantly.
In the following we allocate a fully connected layer
and then use its parameters specifically
to set those of another layer.
Here we need to run the forward propagation
`net(X)` before accessing the parameters.

In [ ]:
%%tab mxnet
net = nn.Sequential()
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Dense(8, activation='relu')
net.add(nn.Dense(8, activation='relu'),
        shared,
        nn.Dense(8, activation='relu', params=shared.params),
        nn.Dense(10))
net.initialize()

X = np.random.uniform(size=(2, 20))

net(X)
# Check whether the parameters are the same
print(net[1].weight.data()[0] == net[2].weight.data()[0])
net[1].weight.data()[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[1].weight.data()[0] == net[2].weight.data()[0])

In [ ]:
%%tab pytorch
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

In [ ]:
%%tab tensorflow
# tf.keras behaves a bit differently. It removes the duplicate layer
# automatically
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# Check whether the parameters are different
print(len(net.layers) == 3)

In [ ]:
%%tab jax
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.Dense(8)
net = nn.Sequential([nn.Dense(8), nn.relu,
                     shared, nn.relu,
                     shared, nn.relu,
                     nn.Dense(1)])

params = net.init(jax.random.PRNGKey(d2l.get_seed()), X)

# Check whether the parameters are different
print(len(params['params']) == 3)

This example shows that the parameters
of the second and third layer are tied.
They are not just equal, they are
represented by the same exact tensor.
Thus, if we change one of the parameters,
the other one changes, too.

:begin_tab:`mxnet, pytorch, tensorflow`
You might wonder,
when parameters are tied
what happens to the gradients?
Since the model parameters contain gradients,
the gradients of the second hidden layer
and the third hidden layer are added together
during backpropagation.
:end_tab:


## Summary

We have several ways of accessing and tying model parameters.


## Exercises

1. Use the `NestMLP` model defined in :numref:`sec_model_construction` and access the parameters of the various layers.
1. Construct an MLP containing a shared parameter layer and train it. During the training process, observe the model parameters and gradients of each layer.
1. Why is sharing parameters a good idea?

:begin_tab:`mxnet`
[Discussions](https://discuss.d2l.ai/t/56)
:end_tab:

:begin_tab:`pytorch`
[Discussions](https://discuss.d2l.ai/t/57)
:end_tab:

:begin_tab:`tensorflow`
[Discussions](https://discuss.d2l.ai/t/269)
:end_tab:

:begin_tab:`jax`
[Discussions](https://discuss.d2l.ai/t/17990)
:end_tab: